# Engineering Notebook: Design Exercise 1

### Ideas for the Wire Protocol

- Use TCP/IP as template
    - Do we need IP addresses in the header?
    - How can we verify packets?
- VBS: Should we use some kind of compression for the message contents?
    - VBS: Maybe not, let's keep it simple
- Header (8 bytes):
    - Version (1 byte)
    - Request code (think: HTTP status code but also use this to specify the operation) (1 byte)
    - Flags/status (2 bytes) -> for the future, also padding
    - Packet checksum (2 bytes)
    - Payload length (2 bytes)

- What type of data are we sending across?
    - Operations + arguments (e.g. page/pagination, optional arguments)
    - Strings (e.g. name, password hash, error messages)
    - Status code (from the server to detect errors)
    - Arrays of strings -> could be sent as string of strings separated by commas like a CSV
        - VBS: we need to allow for commas in a string, should we just use ascii and then have a special non-ascii character for either both end of string and end of item or a separate one for end of item and end of string, kind of like '\0' in C?
        - Let's not use this. Let's use another, more flexible encoding instead
    - Assuming that we never need to send over dictionaries but at most lists, we can use an encoding for "fields" like this:
        - Field ID (1 byte), Field Length (2 bytes), Field Value (bytes)
        - This would mean that the client and server need to agree on which field ID corresponds to which field (e.g. username -> ID 1 etc.), and -- importantly -- what the *type* of each field is (e.g. float, string, unsigned int/int, list of float/string etc.); also, there are only 2^8 = 255 possible fields we can use which should be enough for this application but may cause issues in the future if there is a lot of data that we need to transmit
            - One possible workaround for this is to have an "OTHER" value kind of like in UTF-8 where if the first bit is set, you actually read 2 bytes for the field ID and use that as your index; this would be much more flexible but also more work to implement
        - Do we want to allow lists of different types? Probably not necessary
        - Side note: we can store the types efficiently using some byte flag for "list" and then different byte values for the type, and then check if "list & type" is 1 to detect that it's a list
        - We should agree that we're using little-endian for all communication
            - Is there any way to define flexible length types, e.g. specify "int" but then allow 8-byte integers?


- How do you specify which messages you want to delete?
    - Could go by message ID individually
    - Could go by sender/receiver name

### General Notes
- Each request should carry a token to indicate that the user is logged in and to map the request to the correct account. Do we refresh this token? How often? Does the server just send back the new token with each request? In that case, what do we do if a packet is lost; this would automatically log the user out

### User Interface Design

- How do we display the unread messages?
    - VBS: Maybe users want to be able to open the app without reading *any* messages, so we should only load messages on request
    - We can either group them by conversation or just give a stream of messages ordered by timestamp
    - Think: "Refresh mail" button in any standard Mail program
- To send a message, should users be able to click on a conversation and send a message there or do we just have two input fields, one for the user name and one for the message, and then you just have to input the user name every time

### Data Storage
- How do we want to store the data? Should we just keep stuff in memory, should we use an actual database (e.g. sqlite) or just keep stuff in one or multiple JSON files?

## Code Ideas

Some ideas for what the field ID to field mapping could look like so that the data becomes a bit easier to use for the programmer and we don't need to know all of the field IDs by heart:

In [ ]:
key_to_field_id = {
    "username": 0,
    "password_digest": 1,
}

key_to_field_type = {
    "username": str,
    "password_digest": str,
    "asdkjh": int,
}

def get(data, field: str, default = None):
    if field in key_to_field_id:
        return data[key_to_field_id[id]]
    
    return default

In [ ]:
0x20 -> list
0x21 -> list of int
0x22 -> list of str

In [ ]:
class Message:
    id: int
    content: str
    from_id: int
    to_id: int
    timestamp: datetime

In [ ]:
{
    "messages": [
        {
            "id": 1,
            "content": "My message",
            "from_id": 2,
            "to_id": 3,
            "timestamp": "2025-01-01"
        }
    ],
    "users": [
        {
            "id": 
        }
    ]
}

In [ ]:
def send(data):
    

In [ ]:
send("message", {"to": "test_user", "message": "hello"})

In [ ]:
"hello"

message -> 3
field_type -> str
field_length -> 5